Решение нелинейных уравнений численными методами

Целью данной работы является исследование и реализация численных методов решения нелинейных уравнений, а именно:

- метода половинного деления;

- метода простой итерации;

- метода Ньютона;

- модифицированного метода Ньютона.

Рассматриваются следующие нелинейные уравнения:

$$
(x-3)\cos x = 1, \quad -2\pi \le x \le 2\pi
$$

$$
x \cdot 2^x = 1
$$

In [45]:
import math

EPS = 1e-10
MAX_ITER = 1000

In [46]:
"""
Bisection
"""

def bisection(f, a, b, eps=EPS):
    if f(a) * f(b) >= 0:
        raise ValueError("На отрезке нет корня или их чётное число")

    for i in range(MAX_ITER):
        c = (a + b) / 2
        if abs(f(c)) < eps or abs(b - a) < eps:
            return c, i + 1

        if f(a) * f(c) < 0:
            b = c
        else:
            a = c

    return c, MAX_ITER

In [47]:
"""
Simple iteration
"""

def simple_iteration(phi, x0, eps=EPS):
    x_prev = x0

    for i in range(MAX_ITER):
        x = phi(x_prev)
        if abs(x - x_prev) < eps:
            return x, i + 1
        x_prev = x

    return x, MAX_ITER

In [48]:
"""
Newton
"""

def newton(f, df, x0, eps=EPS):
    x = x0

    for i in range(MAX_ITER):
        x_new = x - f(x) / df(x)
        if abs(x_new - x) < eps:
            return x_new, i + 1
        x = x_new

    return x, MAX_ITER

In [49]:
def modified_newton(f, df, x0, eps=EPS):
    x = x0
    dfx0 = df(x0)

    for i in range(MAX_ITER):
        x_new = x - f(x) / dfx0
        if abs(x_new - x) < eps:
            return x_new, i + 1
        x = x_new

    return x, MAX_ITER

In [50]:
"""
12.3 g) (x-3) cosx = 1, -2pi <= x <= 2pi
"""

def f1(x):
    return (x - 3) * math.cos(x) - 1

def df1(x):
    return math.cos(x) - (x - 3) * math.sin(x)

def phi1_error(x):
    return 3 + 1 / math.cos(x)

def phi1(x, lam=0.2):
    return x - lam * ((x - 3) * math.cos(x) - 1)

a = -2 * math.pi
b =  2 * math.pi

root_bis, it_bis = bisection(f1, a, b)
root_it, it_it = simple_iteration(phi1, 5)
root_new, it_new = newton(f1, df1, 5)
root_mnew, it_mnew = modified_newton(f1, df1, 5)

print("Бисекция (половинное деление)", root_bis, it_bis)
print("Метод простых итераций", root_it, it_it)
print("Метод Ньютона", root_new, it_new)
print("Модифицированный метод Ньютона", root_mnew, it_mnew)

Бисекция (половинное деление) 5.187232027800732 37
Метод простых итераций 5.1872320276897055 33
Метод Ньютона 5.187232027772318 4
Модифицированный метод Ньютона 5.187232027768162 10


In [51]:
"""
12.3 l) x 2^x = 1
"""

def f2(x):
    return x * 2**x - 1

def df2(x):
    return 2**x + x * 2**x * math.log(2)

def phi2(x):
    return 1 / (2**x)

root_bis, it_bis = bisection(f2, 0, 1)
root_it, it_it = simple_iteration(phi2, 0.5)
root_new, it_new = newton(f2, df2, 0.5)
root_mnew, it_mnew = modified_newton(f2, df2, 0.5)

print("Бисекция (половинное деление)", root_bis, it_bis)
print("Метод простых итераций", root_it, it_it)
print("Метод Ньютона", root_new, it_new)
print("Модифицированный метод Ньютона", root_mnew, it_mnew)

Бисекция (половинное деление) 0.6411857445491478 33
Метод простых итераций 0.6411857444848966 28
Метод Ньютона 0.641185744504986 5
Модифицированный метод Ньютона 0.641185744501609 14


Нахождение точек пересечения кривых

In [52]:
def simple_iteration_system(phi, x0, y0, eps):
    x, y = x0, y0

    for i in range(MAX_ITER):
        x_new, y_new = phi(x, y)

        if max(abs(x_new - x), abs(y_new - y)) < eps:
            return x_new, y_new, i + 1

        x, y = x_new, y_new

    return x, y, MAX_ITER

def newton_system(F, J, x0, y0, eps):
    x, y = x0, y0

    for i in range(MAX_ITER):
        f1, f2 = F(x, y)
        J11, J12, J21, J22 = J(x, y)

        det = J11 * J22 - J12 * J21
        if abs(det) < 1e-14:
            raise ValueError("Якобиан вырожден")

        dx = (-f1 * J22 + f2 * J12) / det
        dy = (-J11 * f2 + J21 * f1) / det

        x_new = x + dx
        y_new = y + dy

        if max(abs(dx), abs(dy)) < eps:
            return x_new, y_new, i + 1

        x, y = x_new, y_new

    return x, y, MAX_ITER

In [53]:
"""
12.4 g)
sin(x+2) - y = 1.5
x + cos(y - 2) = 0.5
"""

def phi_system_1(x, y):
    x_new = 0.5 - math.cos(y - 2)
    y_new = math.sin(x + 2) - 1.5
    return x_new, y_new

EPS = 1e-3

x_si, y_si, it_si = simple_iteration_system(
    phi_system_1, 0.0, -1.0, EPS
)

def F1(x, y):
    f1 = math.sin(x + 2) - y - 1.5
    f2 = x + math.cos(y - 2) - 0.5
    return f1, f2

def J1(x, y):
    J11 = math.cos(x + 2)
    J12 = -1
    J21 = 1
    J22 = -math.sin(y - 2)
    return J11, J12, J21, J22

x_n, y_n, it_n = newton_system(
    F1, J1, 0.0, -1.0, EPS
)

print("Метод простых итераций", x_si, y_si, it_si)
print("Метод Ньютона", x_n, y_n, it_n)

Метод простых итераций 1.3463690318861814 -1.7041171989841395 18
Метод Ньютона 1.3463376449218438 -1.7033174842519292 5


In [54]:
"""
12.6 g)
sinx - y = 1.32
cosy - x = -0.85
"""

EPS = 1e-5

def phi_system_2(x, y):
    x_new = math.cos(y) + 0.85
    y_new = math.sin(x) - 1.32
    return x_new, y_new

x_si, y_si, it_si = simple_iteration_system(
    phi_system_2, 0.0, -1.0, EPS
)

def F2(x, y):
    f1 = math.sin(x) - y - 1.32
    f2 = math.cos(y) - x + 0.85
    return f1, f2

def J2(x, y):
    J11 = math.cos(x)
    J12 = -1
    J21 = -1
    J22 = -math.sin(y)
    return J11, J12, J21, J22

x_n, y_n, it_n = newton_system(
    F2, J2, 0.0, -1.0, EPS
)

print("Метод простых итераций", x_si, y_si, it_si)
print("Метод Ньютона", x_n, y_n, it_n)

Метод простых итераций 1.7913377256078902 -0.34422105380937384 12
Метод Ньютона 1.7913386099641768 -0.3442210364060208 12
